In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.5
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


# Way 1 

In [2]:
data = sc.textFile('/labs/laba01/ml-100k/u.data').map(lambda x: x.split("\t"))

df = data.map(lambda x : (x[0], x[1], x[2], x[3]))\
           .toDF(('user id', 'item id', 'rating', 'timestamp'))

In [3]:
df.show(5)

+-------+-------+------+---------+
|user id|item id|rating|timestamp|
+-------+-------+------+---------+
|    196|    242|     3|881250949|
|    186|    302|     3|891717742|
|     22|    377|     1|878887116|
|    244|     51|     2|880606923|
|    166|    346|     1|886397596|
+-------+-------+------+---------+
only showing top 5 rows



In [4]:
df_302 = df.filter(df['item id'] == 302)\
             .groupBy([df['item id'], df['rating']])\
             .count()\
             .sort(df['rating'], ascending=True)
df_302 = df_302.collect()

In [5]:
df_all = df.groupBy(df['rating'])\
           .count()\
           .sort(df['rating'], ascending=True)
df_all = df_all.collect()

In [6]:
json_ = {"hist_film": [df_302[i][2] for i in range(len(df_302))],
         "hist_all": [df_all[i][1] for i in range(len(df_all))]
         }

In [7]:
json_

{'hist_film': [2, 10, 46, 119, 120],
 'hist_all': [6110, 11370, 27145, 34174, 21201]}

In [8]:
import json

with open("lab01.json", "w", encoding="utf-8") as file:
    json.dump(json_, file)

## Way 2

In [9]:
df = data

In [10]:
df_302 = df.filter(lambda x: x[1]=='302')\
           .flatMap(lambda x: [x[2]])\
           .sortBy(lambda x: x[0])\
           .countByKey()

In [11]:
df_all = df.flatMap(lambda x: [x[2]])\
           .sortBy(lambda x: x[0])\
           .countByKey() 

In [12]:
json_ = {"hist_film": list(df_302.values()),
         "hist_all": list(df_all.values())
         }

In [13]:
json_

{'hist_film': [2, 10, 46, 119, 120],
 'hist_all': [6110, 11370, 27145, 34174, 21201]}

In [14]:
sc.stop()